In [1]:
import vtk

In [2]:
filename = "PLY/target.ply"
 
sphereSource = vtk.vtkSphereSource()
sphereSource.Update()
 
plyWriter = vtk.vtkPLYWriter()
plyWriter.SetFileName(filename)
plyWriter.SetInputConnection(sphereSource.GetOutputPort())
plyWriter.Write()
 
#Read and display for verication
reader = vtk.vtkPLYReader()
reader.SetFileName(filename)
reader.Update()
 
mapper = vtk.vtkPolyDataMapper()
mapper.SetInputConnection(reader.GetOutputPort())
 
actor = vtk.vtkActor()
actor.SetMapper(mapper)
 
renderer = vtk.vtkRenderer()
renderWindow = vtk.vtkRenderWindow()
renderWindow.AddRenderer(renderer)
renderWindowInteractor = vtk.vtkRenderWindowInteractor()
renderWindowInteractor.SetRenderWindow(renderWindow)
 
renderer.AddActor(actor)
renderer.SetBackground(.3, .6, .3)   #Background color green
 
renderWindow.Render()
renderWindowInteractor.Start()

In [13]:
from numpy import pi, sin, cos, mgrid
dphi, dtheta = pi/250.0, pi/250.0
[phi,theta] = mgrid[0:pi+dphi*1.5:dphi,0:2*pi+dtheta*1.5:dtheta]
m0 = 4; m1 = 3; m2 = 2; m3 = 3; m4 = 6; m5 = 2; m6 = 6; m7 = 4;
r = sin(m0*phi)**m1 + cos(m2*phi)**m3 + sin(m4*theta)**m5 + cos(m6*theta)**m7
x = r*sin(phi)*cos(theta)
y = r*cos(phi)
z = r*sin(phi)*sin(theta)

In [14]:
from mayavi import mlab

In [15]:
x

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  2.51217719e-02,   2.50488973e-02,   2.48367387e-02, ...,
          2.50488973e-02,   2.51217719e-02,   2.50488973e-02],
       [  5.01904937e-02,   5.00447598e-02,   4.96204877e-02, ...,
          5.00447598e-02,   5.01904937e-02,   5.00447598e-02],
       ..., 
       [  2.51185842e-02,   2.50457098e-02,   2.48335519e-02, ...,
          2.50457098e-02,   2.51185842e-02,   2.50457098e-02],
       [  2.44929360e-16,   2.44219138e-16,   2.42151484e-16, ...,
          2.44219138e-16,   2.44929360e-16,   2.44219138e-16],
       [ -2.51217719e-02,  -2.50488973e-02,  -2.48367387e-02, ...,
         -2.50488973e-02,  -2.51217719e-02,  -2.50488973e-02]])

In [3]:
s = mlab.mesh(x, y, z)

In [4]:
mlab.show()

In [1]:
from plyfile import PlyData, PlyElement

In [2]:
data = PlyData.read('PLY/target.ply')

In [3]:
data

PlyData((PlyElement('vertex', (PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float'), PlyProperty('texture_u', 'float'), PlyProperty('texture_v', 'float')), count=22474, comments=[]), PlyElement('face', (PlyListProperty('vertex_indices', 'uchar', 'int'),), count=43198, comments=[])), text=False, byte_order='<', comments=[u'VCGLIB generated'], obj_info=[])

In [4]:
X=data['vertex']['x']

In [5]:
Y=data['vertex']['y']

In [6]:
Z=data['vertex']['z']

In [7]:
from mayavi import mlab

In [8]:
s = mlab.plot3d(X,Y,Z,colormap='Spectral')

In [ ]:
mlab.show()